In [1]:
### Setting Things UP
# Imports
import glob
import os
import re
import astropy.table
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import the data from the LENSS_TSL_0008 folder and format into a Pandas dataframe

In [3]:
# Get the file names and printf first few lines
datafolder = '/content/drive/My Drive/UChicago Undergrad/Second Year/Metcalf GLAS Project/Data/Sensors Data/LENSS_TSL_0008/LENS/_T/'
directory_files = os.listdir(datafolder)

df = pd.DataFrame()
for file in directory_files:
    df_file = pd.read_csv(os.path.join(datafolder, file), low_memory=False, sep=";", names = ['Time (UTC)','Time (CST)','Temperature','Frequency','Voltage','Sensor'])
    df = pd.concat([df, df_file], ignore_index=True)

In [ ]:
import datetime
from datetime import date, timedelta

# Assuming the timestamp column is stored in a variable called "timestamp_str"
timestamp_format = '%Y-%m-%dT%H:%M:%S.%f'  # Define the format of the timestamp

df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%dT%H:%M:%S.%f')
df["Time (UTC)"] = pd.to_datetime(df['Time (UTC)'], format='%Y-%m-%dT%H:%M:%S.%f')

df["Day"] = df["Time (UTC)"].dt.day
df["Month"] = df["Time (UTC)"].dt.month
df["Year"] = df["Time (UTC)"].dt.year

df["Date"] = df["Time (UTC)"].apply(lambda x: x.date())

In [ ]:
sdate, edate = (df['Date'].min(), df['Date'].max())
all_dates = [sdate+timedelta(days=x) for x in range((edate-sdate).days)]

Remove days/times where the sky is not clear

In [ ]:
import requests

date = '2023-03-27'
lat = '37.7749'  # latitude of the location
lon = '-122.4194'  # longitude of the location

#url = f'https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={date}&appid={api_key}'
#url = f'http://api.openweathermap.org/data/2.5/forecast?id=524901&appid={api_key}'

url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=current,minutely,daily,alerts&dt={date}&appid={api_key}'

response = requests.get(url)

# check that response was successful
if response.status_code == 200:
    data = response.json()
else:
    print(f'Request failed with status code {response.status_code}')

Request failed with status code 401


In [ ]:
data

In [ ]:
if 'hourly' in data:
    hourly_data = data['hourly']
    cloud_cover = sum(hourly['clouds'] for hourly in hourly_data) / len(hourly_data)  # calculate average cloud cover
    
    if cloud_cover < 10:  # check if cloud cover is below 10%
        print('The sky was clear')
    else:
        print('The sky was not clear')
else:
    print('No hourly data found')

Find the moon illumination

In [ ]:
%pip install astroplan

In [ ]:
from astroplan import Observer
import astropy.units as astro_units
from astropy.time import Time

In [ ]:
SENSOR_LONGITUDE = -88.5375518*astro_units.deg
SENSOR_LATITUDE = 42.587277*astro_units.deg
SENSOR_ELEVATION = 879*astro_units.m
sensor_location = Observer(longitude=SENSOR_LONGITUDE, latitude=SENSOR_LATITUDE, elevation=SENSOR_ELEVATION)

In [ ]:
dates_seen = {}
moon_values = []
for i, row in df.iloc[:1000, :].iterrows():
  timestamp = row["Time (UTC)"]
  evening_date = str(timestamp.date())
  evening_twilight = str(evening_date) + ' 16:00:00.000'
  
  dates_seen[evening_date] = dates_seen.get(evening_date, float(str(sensor_location.moon_illumination(evening_twilight))))
  moon_values.append(dates_seen[evening_date])

dates_seen

{'2022-08-23': 0.12448116142774585}